# Singleton

## implementation

Since there is no concept of private members as for c++, we have to take advantage of its dict mechanism to create singletons.

In [1]:
# implementation of singleton

class Singleton(object):
    def __new__(cls):
        if not hasattr(cls, 'my_instance'): # we use a field to mark the instance of the Singleton object
            cls.my_instance = super(Singleton, cls).__new__(cls)
        return cls.my_instance

In [2]:
s = Singleton
print(vars(s)) # there is no 'my_instance' in the dict mapping since we haven't instantiate the object

s1 = Singleton()
s2 = Singleton()

print(f"s1 address: {id(s1)}, s2 address: {id(s2)}")

s1.variable = 10
print(f"s2 variable: {s2.variable}")

print(vars(s)) # the'instance' appears after instantiation

{'__module__': '__main__', '__firstlineno__': 3, '__new__': <staticmethod(<function Singleton.__new__ at 0x7f60bc220720>)>, '__static_attributes__': (), '__dict__': <attribute '__dict__' of 'Singleton' objects>, '__weakref__': <attribute '__weakref__' of 'Singleton' objects>, '__doc__': None}
s1 address: 140053449807184, s2 address: 140053449807184
s2 variable: 10
{'__module__': '__main__', '__firstlineno__': 3, '__new__': <staticmethod(<function Singleton.__new__ at 0x7f60bc220720>)>, '__static_attributes__': (), '__dict__': <attribute '__dict__' of 'Singleton' objects>, '__weakref__': <attribute '__weakref__' of 'Singleton' objects>, '__doc__': None, 'my_instance': <__main__.Singleton object at 0x7f60bc201550>}


## borg/monostate singleton

Borg singleton is a design pattern in Python that allows state sharing for different instances.

In [4]:
# the child object is the same as its parent singleton

class ChildSingleton(Singleton):
    pass

child_s = ChildSingleton()

print(f"singleton address: {id(s1)}, child address: {id(child_s)}")

# we change the variable value of object s1, this has the print out put to 20
s1.variable = 20 
print(f"child variable: {child_s.variable}")

singleton address: 140053449807184, child address: 140053449807184
child variable: 20


In [102]:
# borg
# This allows different object has the same state

class BorgSingleton(object):
    # this allows all children share the same dict mapping
    _shared_borg_state = {}

    def __new__(cls):
        obj = super(BorgSingleton, cls).__new__(cls)
        obj.__dict__ = cls._shared_borg_state
        return obj # return object instead of instance

class ChildBorg(BorgSingleton):
    pass

borg = BorgSingleton()
child_borg = ChildBorg()

borg.variable = 40
print(f"borg address: {id(borg)}, child address: {id(child)}")
print(f"child variable: {child_borg.variable}")

borg address: 140195608933920, child address: 140195597549648
child variable: 40


## thread safe

In [75]:
import threading

class SafeSingleton(object):

    _lock = threading.Lock() # for amotic lock
    
    def __new__(cls):
        with cls._lock:
            if not hasattr(cls, 'instance'):
                cls.instance = super(SafeSingleton, cls).__new__(cls)
        return cls.instance

        
s = SafeSingleton
print(s.__dict__)

s1 = SafeSingleton()
s2 = SafeSingleton()

print(f"s1 address: {id(s1)}, s2 address: {id(s2)}")
print(s.__dict__)

{'__module__': '__main__', '__firstlineno__': 3, '_lock': <unlocked _thread.lock object at 0x7f81d4ca0c40>, '__new__': <staticmethod(<function SafeSingleton.__new__ at 0x7f81d46d3600>)>, '__static_attributes__': (), '__dict__': <attribute '__dict__' of 'SafeSingleton' objects>, '__weakref__': <attribute '__weakref__' of 'SafeSingleton' objects>, '__doc__': None}
s1 address: 140195608925520, s2 address: 140195608925520
{'__module__': '__main__', '__firstlineno__': 3, '_lock': <unlocked _thread.lock object at 0x7f81d4ca0c40>, '__new__': <staticmethod(<function SafeSingleton.__new__ at 0x7f81d46d3600>)>, '__static_attributes__': (), '__dict__': <attribute '__dict__' of 'SafeSingleton' objects>, '__weakref__': <attribute '__weakref__' of 'SafeSingleton' objects>, '__doc__': None, 'instance': <__main__.SafeSingleton object at 0x7f81d5781550>}


## c++ style

As said before, there is no private member for python, we can then disable the init function and only allow a static instance method.


In [11]:
class Singleton:
    def __init__(cls):
        raise NotImplementedError
        # etc
    
    @staticmethod
    def instance():
        if not hasattr(Singleton, 'instance'):
            Singleton.instance = Singleton()
        return Singleton.instance

try:
    s = Singleton()
except:
    print("Error: not implemented")
s1 = Singleton.instance()
s2 = Singleton.instance()
print(f"s1 address: {id(s1)}, s2 address: {id(s2)}")

Error: not implemented
s1 address: 140053325823360, s2 address: 140053325823360


## decorator

In [14]:
class Singleton:
    def __init__(self, cls):
        self._cls = cls

    def instance(self):
        try:
            return self._instance
        except AttributeError:
            self._instance = self._cls()
            return self._instance

    def __call__(self):
        raise TypeError('Singletons must be accessed through `instance()`.')

    def __instancecheck__(self, inst):
        return isinstance(inst, self._cls)

@Singleton
class DBConnection(object):
    def __init__(self):
        """Initialize your database connection here."""
        pass

    def __str__(self):
        return 'Database connection object'

try:
    s =  DBConnection()
except:
    print("Error: forbidden")
s1 = DBConnection.instance()
s2 = DBConnection.instance()
print(f"s1 address: {id(s1)}, s2 address: {id(s2)}")

Error: forbidden
s1 address: 140053324125856, s2 address: 140053324125856


## metaclass

In [97]:
class SingletonMeta(type):
    _instance = {}
    def __call__(cls, *args, **kwargs):
        if not cls in cls._instance:
            cls._instance[cls] = super(SingletonMeta, cls).__call__(*args, **kwargs)
        return cls._instance[cls]

class Singleton(metaclass=SingletonMeta):
    pass

In [99]:
# Test the singleton class
# Create two instances of SingletonClass
s1 = Singleton()
s2 = Singleton()
# Check if both instances refer to the same object
print(f"s1 address: {id(s1)}, s2 address: {id(s2)}")

s1 address: 140195436725984, s2 address: 140195436725984


## common applications

Logging: The Python logging module is implemented as a Singleton so that all modules in an application share a common logging instance.

Database connection management: As discussed in this article, many web frameworks, such as Django, use the Singleton pattern to ensure that only one instance of a database connection exists at any given time.

Configuration management: Configuration data for an application is often stored in a Singleton object so that it can be easily accessed by all parts of the application.

In [16]:
# example of sql

# to install the package: pip install mysql-connector-python

import mysql.connector

class MysqlConnectionSingleton(object):
    def __new__(cls, *args, **kwargs):
        if not hasattr(MysqlConnectionSingleton, 'instance'):
            cls.instance = super(MysqlConnectionSingleton, cls).__new__(cls)
        return cls.instance

    def __init__(self, host, user, pw, db):
        self._host = host
        self._user = user
        self._password = pw
        self._database = db
        self._connection = None
        print(f"Connection:\n host: {self._host}\n user: {self._user}\n database: {self._database}")

        if not self._connection:
            self._connection = "connect to localhost"

    def get_connection(self):
        return self._connection

    def query(self, q):
        try:
            connection = self.get_connection()
            cursor = connection.cursor()
            cursor.execute(q)
            result = cursor.fetchall()
            return result
        except mysql.connector.Error as err:
            print(f"Error: {err}")
        finally:
            cursor.close()

# usage
myconnection1 = MysqlConnectionSingleton("localhost", "me", "mypw", "test")
myconnection2 = MysqlConnectionSingleton("localhost", "me", "mypw", "test")
print(myconnection1.get_connection() == myconnection2.get_connection())

#query = "SELECT * FROM your_table LIMIT 5"

# Executing the query
#result = myconnection.execute_query(query)

Connection:
 host: localhost
 user: me
 database: test
Connection:
 host: localhost
 user: me
 database: test
True
